In [1]:
from pyimagesearch.alignment import align_images
from collections import namedtuple
import pytesseract
import argparse
import imutils
import cv2
from  matplotlib import pyplot as plt
import os

In [2]:
def cleanup_text(text):
    # strip out non-ASCII text so we can draw the text on the image
    # using OpenCV
    return "".join([c if ord(c) < 128 else "" for c in text]).strip()

In [3]:
OCRLocation = namedtuple("OCRLocation", ["id", "bbox","filter_keywords"])

In [4]:
OCR_LOCATIONS = [
    OCRLocation("step1_first_name", (195, 1072, 426, 50),
        ["middle", "initial", "first", "name"]),
    OCRLocation("Phone_O", (195, 1128, 222, 47),
        ["number"]),
    OCRLocation("Phone_H", (420, 1121, 244, 52),
        ["number"]),
    OCRLocation("cellphone", (934, 1122, 245, 55),
        ["number"]),
    OCRLocation("sign", (256, 1222, 215, 74),
        ["number"]),
]

In [5]:
print("[INFO] loading images...")
image = cv2.imread("Test.png")
template = cv2.imread("Form.jpg")
#plt.imshow(image)
#plt.imshow(template)

[INFO] loading images...


In [6]:
print("[INFO] aligning images...")
aligned = align_images(image, template)

[INFO] aligning images...


In [7]:
print("[INFO] OCR'ing document...")
parsingResults = []

[INFO] OCR'ing document...


In [8]:
j=0
save_filename='image'
data='output'
def saveImage(image,SaveAddress,num):
    address= SaveAddress+data+'_'+str(num).zfill(4)+'.jpg'
    cv2.imwrite(address,image)

In [9]:
for loc in OCR_LOCATIONS:
# extract the OCR ROI from the aligned image
    #print(loc)
    (x, y, w, h) = loc.bbox
    roi = aligned[y:y + h, x:x + w]
# OCR the ROI using Tesseract
    rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    j+=1
    saveImage(rgb,save_filename,j)
    #text = pytesseract.image_to_string(rgb)

In [10]:
#(x, y, w, h) = OCR_LOCATIONS[2].bbox
#print((x, y, w, h))
#roi = aligned[y:y + h, x:x + w]
#rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
#text = pytesseract.image_to_string(rgb)
#print(text)
#plt.imshow(rgb)

In [23]:
import argparse
import json
from typing import Tuple, List

import cv2
import editdistance
from path import Path

from tensorflow.python.framework import ops
from tensorflow import keras

from SimpleHTR.src.dataloader_iam import DataLoaderIAM, Batch
from SimpleHTR.src.model import Model, DecoderType
from SimpleHTR.src.preprocessor import Preprocessor

In [24]:
class FilePaths:
    """Filenames and paths to data."""
    fn_char_list = 'charList.txt'
    fn_summary = 'summary.json'
    fn_corpus = 'corpus.txt'

In [25]:
def get_img_height() -> int:
    """Fixed height for NN."""
    return 32

In [26]:
def get_img_size(line_mode: bool = False) -> Tuple[int, int]:
    """Height is fixed for NN, width is set according to training mode (single words or text lines)."""
    if line_mode:
        return 256, get_img_height()
    return 128, get_img_height()

In [27]:
def write_summary(char_error_rates: List[float], word_accuracies: List[float]) -> None:
    """Writes training summary file for NN."""
    with open(FilePaths.fn_summary, 'w') as f:
        json.dump({'charErrorRates': char_error_rates, 'wordAccuracies': word_accuracies}, f)

In [30]:
def infer(model: Model, fn_img: Path) -> None:
    """Recognizes text in image provided by file path."""
    img = cv2.imread(fn_img, cv2.IMREAD_GRAYSCALE)
    assert img is not None
    preprocessor = Preprocessor(get_img_size(), dynamic_width=True, padding=16)
    img = preprocessor.process_img(img)
    batch = Batch([img], None, 1)
    recognized, probability = model.infer_batch(batch, True)
    print(f'Recognized: "{recognized[0]}"')
    print(f'Probability: {probability[0]}')

In [34]:
ops.reset_default_graph()
decoder_mapping = {'bestpath': DecoderType.BestPath,
                   'beamsearch': DecoderType.BeamSearch,
                   'wordbeamsearch': DecoderType.WordBeamSearch}
decoder_type = decoder_mapping['bestpath']
model = Model(list(open(FilePaths.fn_char_list).read()), decoder_type, must_restore=True, dump='store_true')
for i in range(5):
    test_image='C:/Users/e1222/Documents/ocr-document/ocr-document/image/imageoutput_%d.jpg'%(i)
    print(test_image)
    infer(model, test_image)

Python: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
Tensorflow: 2.6.0
Init with stored values from snapshot-13
INFO:tensorflow:Restoring parameters from snapshot-13
C:/Users/e1222/Documents/ocr-document/ocr-document/image/imageoutput_0.jpg
Write dump of NN to file: ../dump/rnnOutput_0.csv
Recognized: "move"
Probability: 0.3282751739025116
C:/Users/e1222/Documents/ocr-document/ocr-document/image/imageoutput_1.jpg
Write dump of NN to file: ../dump/rnnOutput_0.csv
Recognized: "" . nomiuating"
Probability: 0.13835062086582184
C:/Users/e1222/Documents/ocr-document/ocr-document/image/imageoutput_2.jpg
Write dump of NN to file: ../dump/rnnOutput_0.csv
Recognized: "orFam"
Probability: 0.00498551269993186
C:/Users/e1222/Documents/ocr-document/ocr-document/image/imageoutput_3.jpg
Write dump of NN to file: ../dump/rnnOutput_0.csv
Recognized: "tomorrew"
Probability: 0.21413789689540863
C:/Users/e1222/Documents/ocr-document/ocr-document/image/imageoutput_4.jpg
Write dump of N